In [316]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib as plt
from torch.utils.data import DataLoader
from sklearn.model_selection import StratifiedKFold
import numpy as np

In [317]:

class BinaryClassifier(nn.Module):
    def __init__(self, input_dim, hidden_layers):
        super(BinaryClassifier, self).__init__()
        layers = []
        #add fully connected layers with the input dim dividing by two each time the input shape
        output_dim = max(input_dim//(2), 1)
        for i in range(hidden_layers):
            i += 1
            layers.append(nn.Linear(input_dim,output_dim))
            layers.append(nn.ReLU())
            input_dim = output_dim
            output_dim = max(int(input_dim//(1.2*i)), 1)
        #add final layer with sigmiod output and final linear layer
        layers.append(nn.Linear(input_dim,1))
        layers.append(nn.Sigmoid())

        #create a Seqeuntial implementation, no additional layers but provides one line to run through all layers
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        #apply the neural net
        return self.network(x)

In [318]:
#input shape is not correct
input_shape = 30
hidden_layers = 5
model_1 = BinaryClassifier(input_shape, hidden_layers)

In [319]:
optimizer = optim.Adam(model_1.parameters(), lr=1e-3)
loss_fn = nn.BCELoss()

In [320]:
from sklearn.model_selection import train_test_split
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset

In [321]:

def dataset_to_numpy(dataset: TensorDataset):
    X, y = dataset.tensors
    # Ensure on CPU and detached from computation graph
    X_np = X.detach().numpy()
    y_np = y.detach().numpy()
    return X_np, y_np


In [322]:
def load_data():
    # Read data
    df = pd.read_csv('br_cancer_normalized.csv', encoding='latin-1')

    # Separate features and labels
    X = df.drop(columns=['Diagnosis']).to_numpy()
    y = df['Diagnosis'].to_numpy()
    return X, y

In [323]:
#import whatever data loader function is needed
#should output train dataloader and test dataloader  as the full train and test datasets
X_np, y_np = load_data()
print(X_np.shape, y_np.shape)

(569, 30) (569,)


In [324]:

def visualize_prediction_distribution(model, dataloader, threshold=0.5):
    """
    Visualize the percentage of predicted 0s and 1s from a binary classifier.
    
    Args:
        model: Trained PyTorch model
        dataloader: DataLoader containing your dataset
        threshold: Decision threshold for classification (default=0.5)
        device: "cpu" or "cuda"
    """
    model.eval()
    preds = []
    
    with torch.no_grad():
        for batch_X, _ in dataloader:
            outputs = model(batch_X).squeeze()
            
            # If model ends with Sigmoid
            if outputs.dim() == 0:
                outputs = outputs.unsqueeze(0)
            
            predicted = (outputs >= threshold).int().cpu().numpy()
            preds.extend(predicted)
    
    preds = np.array(preds)
    total = len(preds)
    percent_0 = (preds == 0).sum() / total * 100
    percent_1 = (preds == 1).sum() / total * 100
    print(percent_0, "this is percent 0")
    # # Bar plot
    # plt.bar(["Predicted 0", "Predicted 1"], [percent_0, percent_1], color=["skyblue", "salmon"])
    # plt.ylabel("Percentage (%)")
    # plt.title("Prediction Distribution")
    # plt.show()
    
    print(f"Predicted 0: {percent_0:.2f}%")
    print(f"Predicted 1: {percent_1:.2f}%")


In [325]:
from statistics import mean, stdev
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn import linear_model
from sklearn import datasets

In [326]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Subset
from sklearn.model_selection import StratifiedKFold
import copy

def test(model, data, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch_X, batch_y in data:
            batch_y = batch_y.unsqueeze(1)
            logits = model(batch_X)
            loss = criterion(logits, batch_y)
            test_loss += loss.item() * batch_X.size(0)

            preds = (torch.sigmoid(logits) >= 0.5).float()
            correct += (preds == batch_y).sum().item()
            total += batch_y.size(0)
    
    avg_loss = test_loss / total
    accuracy = correct / total
    print(f"Test Loss: {avg_loss:.4f} | Test Acc: {accuracy:.4f}")
    return avg_loss, accuracy
    

def train_with_stratified_kfold(X, y, model_fn, input_shape, hidden_layers, batch_size=32, lr=1e-3, k_folds=5, num_epochs=300):
    # Wrap into dataset
    dataset = TensorDataset(torch.tensor(X, dtype=torch.float32),
                            torch.tensor(y, dtype=torch.float32))
    
    skf = StratifiedKFold(n_splits=k_folds, shuffle=True)
    all_fold_results = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        print(f"\n--- Fold {fold+1} ---")

        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)

        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
     # fresh model for each fold
        cur_model = BinaryClassifier(30,5)
        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(cur_model.parameters(), lr=lr)

        for epoch in range(num_epochs):
            cur_model.train()
            epoch_loss = 0.0
            correct = 0
            total = 0

            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = cur_model(batch_X).squeeze()
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

                epoch_loss += loss.item() * batch_X.size(0)
                preds = (torch.sigmoid(outputs) >= 0.5).float()
                correct += (preds == batch_y).sum().item()
                total += batch_y.size(0)

            train_acc = correct / total
            train_loss = epoch_loss / total
            print(f"Epoch {epoch+1}: Train Loss {train_loss:.4f} | Train Acc {train_acc:.4f}")

        avg_loss, accuracy = test(cur_model, val_loader, criterion)
        all_fold_results.append((avg_loss, accuracy))
    visualize_prediction_distribution(cur_model, val_loader)

    
    return all_fold_results


In [327]:
#training with 
fold_results = train_with_stratified_kfold(X_np,y_np,model_1,30,5)


--- Fold 1 ---
Epoch 1: Train Loss 0.7753 | Train Acc 0.3736
Epoch 2: Train Loss 0.7740 | Train Acc 0.3736
Epoch 3: Train Loss 0.7727 | Train Acc 0.3736
Epoch 4: Train Loss 0.7714 | Train Acc 0.3736
Epoch 5: Train Loss 0.7700 | Train Acc 0.3736
Epoch 6: Train Loss 0.7687 | Train Acc 0.3736
Epoch 7: Train Loss 0.7672 | Train Acc 0.3736
Epoch 8: Train Loss 0.7659 | Train Acc 0.3736
Epoch 9: Train Loss 0.7644 | Train Acc 0.3736
Epoch 10: Train Loss 0.7629 | Train Acc 0.3736
Epoch 11: Train Loss 0.7614 | Train Acc 0.3736
Epoch 12: Train Loss 0.7598 | Train Acc 0.3736
Epoch 13: Train Loss 0.7582 | Train Acc 0.3736
Epoch 14: Train Loss 0.7568 | Train Acc 0.3736
Epoch 15: Train Loss 0.7554 | Train Acc 0.3736
Epoch 16: Train Loss 0.7538 | Train Acc 0.3736
Epoch 17: Train Loss 0.7523 | Train Acc 0.3736
Epoch 18: Train Loss 0.7508 | Train Acc 0.3736
Epoch 19: Train Loss 0.7493 | Train Acc 0.3736
Epoch 20: Train Loss 0.7479 | Train Acc 0.3736
Epoch 21: Train Loss 0.7464 | Train Acc 0.3736
Epoch 

AttributeError: module 'matplotlib' has no attribute 'bar'

In [ ]:
print(fold_results)